In [ ]:
from __future__ import division
from pyoperators import pcg
from pysimulators import profile

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
import os

path = os.getcwd()
path_mm = os.path.dirname(path)
import sys
print(path_mm)
sys.path.append(path_mm)


import frequency_acquisition as Acq
#import component_acquisition

# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# FG-Buster packages
import component_model as c
import mixing_matrix as mm

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils

from importlib import reload
from pyoperators import *
from pysimulators import *


from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
warnings.filterwarnings("ignore")

# # %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

dictfilename = 'dicts/pipeline_demo.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)

d['nf_recon'] = 2
d['nf_sub'] = 4
d['nside'] = 128
npix=12*d['nside']**2
d['RA_center'] = 0
d['DEC_center'] = -57
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
d['effective_duration'] = 3
d['npointings'] = 300
d['filter_nu'] = 220 * 1e9
d['photon_noise'] = False
d['noiseless'] = True
d['config'] = 'FI'
d['filter_relative_bandwidth'] = 0.25
d['MultiBand'] = True
d['planck'] = True
d['dtheta'] = 15
d['synthbeam_dtype'] = float
d['debug'] = False

# # %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

### Load the default dictionary and perform the changements
def get_dict(args={}):
    
    ### Get the default dictionary
    dictfilename = 'dicts/pipeline_demo.dict'
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(dictfilename)
    d['npointings'] = 9
    for i in args.keys():
        
        d[str(i)] = args[i]
    
    return d


Nrec = 1
Nsub = 3
npointings = 2000
convolution = True

### Noise
noise = True
level_noise_planck = 1

### Physical band
band = 220
band_planck = 217
relative_bandwidth = 0.25
nside = 256
type_qubic = 'Wide'

In [ ]:
d150 = get_dict({'npointings':npointings, 'nf_recon':Nrec, 'nf_sub':Nsub, 'nside':nside,
              'filter_nu':150 * 1e9, 'noiseless': not noise,'photon_noise':noise, 'nhwp_angles':3, 'effective_duration':3})

d220 = get_dict({'npointings':npointings, 'nf_recon':Nrec, 'nf_sub':Nsub, 'nside':nside,
              'filter_nu':220 * 1e9, 'noiseless': not noise,'photon_noise':noise, 'nhwp_angles':3, 'effective_duration':3})



sky_config = {'cmb':42}

reload(Acq)

qubic_acquisition150 = Acq.QubicIntegrated(d150, Nsub=Nsub, Nrec=1)
qubic_acquisition220 = Acq.QubicIntegrated(d220, Nsub=Nsub, Nrec=1)


if type_qubic == 'Wide':
    myqubic = Acq.QubicWideBand(qubic_acquisition150, qubic_acquisition220)
else:
    myqubic = Acq.QubicTwoBands(qubic_acquisition150, qubic_acquisition220)


s = Acq.Sky(sky_config, myqubic)
#sed = mm.MixingMatrix(*comp).evaluator(s.allnus)#(beta=None)

beta = None
s_nu = s.scale_component(beta)

planck_acquisition143 = Acq.PlanckAcquisition(143, qubic_acquisition150.scene)
planck_acquisition217 = Acq.PlanckAcquisition(217, qubic_acquisition150.scene)

joint = Acq.QubicPlanckMultiBandAcquisition(myqubic, [planck_acquisition143, planck_acquisition217])

In [ ]:
TOD_QUBIC = myqubic.get_TOD(sky_config, beta=None, noise=True)

In [ ]:
p143 = np.mean(s_nu[:Nsub], axis=0).ravel()
p217 = np.mean(s_nu[Nsub:2*Nsub], axis=0).ravel()
n143 = planck_acquisition143.get_noise().ravel()
n217 = planck_acquisition217.get_noise().ravel()

In [ ]:
if type_qubic == 'Wide':
    
    tod = TOD_QUBIC.ravel()
    tod = np.r_[tod, p143+n143]
    tod = np.r_[tod, p217+n217]
    
else:
    TOD_QUBIC_150 = TOD_QUBIC[:992]
    TOD_QUBIC_220 = TOD_QUBIC[992:2*992]
    tod = TOD_QUBIC_150.ravel()
    tod = np.r_[tod, p143+n143]
    tod = np.r_[tod, TOD_QUBIC_220.ravel()]
    tod = np.r_[tod, p217+n217]
tod = np.array(tod)

plt.plot(tod)

In [ ]:
H = joint.get_operator()
invN = joint.get_invntt_operator()

In [ ]:
tod.shape

In [ ]:
H.shapeout

In [ ]:
A = H.T * invN * H
b = H.T * invN * tod

In [ ]:
s=pcg(A, b, disp=True, maxiter=40, x0=None, tol=1e-40)

In [ ]:
plt.figure(figsize=(15, 5))

reso=15

hp.gnomview(s_nu[0, :, 1], rot=center, reso=reso, cmap='jet', sub=(1, 3, 1), min=-8, max=8)
hp.gnomview(s['x'][0, :, 1], rot=center, reso=reso, cmap='jet', sub=(1, 3, 2), min=-8, max=8)
hp.gnomview(s['x'][0, :, 1]-s_nu[0, :, 1], rot=center, reso=reso, cmap='jet', sub=(1, 3, 3), min=-8, max=8)

plt.show()

In [ ]:
plt.figure(figsize=(15, 5))

hp.gnomview(s_nu[0, :, 2], rot=center, reso=reso, cmap='jet', sub=(1, 3, 1), min=-8, max=8)
hp.gnomview(s['x'][1, :, 2], rot=center, reso=reso, cmap='jet', sub=(1, 3, 2), min=-8, max=8)
hp.gnomview(s['x'][1, :, 2]-s_nu[0, :, 2], rot=center, reso=reso, cmap='jet', sub=(1, 3, 3), min=-8, max=8)

plt.show()

In [ ]:
plt.plot(tod_q)
plt.plot(WB.get_noise().ravel())

In [ ]:
H.shapein

In [ ]:
plt.plot(H(m[2:]))

In [ ]:
comp = [c.Dust(nu0=150, temp=20)]

In [ ]:
for i in comp:
    print(i)

In [ ]:
try :
    is_cmb = mm.MixingMatrix(*comp).components.index('CMB')
except:
    is_cmb = False

In [ ]:
is_cmb

In [ ]:
Sky({'cmb':42}, WB).cmb#.generate_sky_nu0(150, {'cmb':42, 'dust':'d0'})

In [ ]:
reload(Acq)

s = Acq.Sky({'cmb':42, 'dust':'d0'}, qubic_acquisition150)
#sed = mm.MixingMatrix(*comp).evaluator(s.allnus)#(beta=None)

beta = np.array([1.54])
s_nu = s.scale_component(beta)

In [ ]:
hp.mollview(s_nu[0, :, 0], cmap='jet', sub=(1, 3, 1), min=-300, max=300)
hp.mollview(s_nu[1, :, 0], cmap='jet', sub=(1, 3, 2), min=-300, max=300)
#hp.mollview(s_nu[2, :, 0], cmap='jet', sub=(1, 3, 3), min=-300, max=300)